# 🚔 DispatchAI: Remote Brain (Colab Server)
**Run this notebook on a T4 GPU Runtime.**
This server acts as the AI processing hub for the EAEDS Control system.

**Stack:**
*   **STT:** `faster-whisper-large-v3`
*   **Emotion:** `opensmile` (Acoustic) + `BERT` (Semantic)
*   **LLM:** `spikecodes/ai-911-operator` (Mistral-7B Adapter)
*   **Tunnel:** `cloudflared` (Reliable & Free)

In [ ]:
# 1. Install Dependencies
!sudo apt-get install -y ffmpeg pciutils
# Installing transformers and specific dependencies for Mistral 4-bit loading
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install git+https://github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/peft
!pip install accelerate bitsandbytes faster-whisper opensmile fastapi uvicorn python-multipart pydub pinggy nest_asyncio
# Download Cloudflare Tunnel
!wget -q -nc https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /tmp/cloudflared
!chmod +x /tmp/cloudflared

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 41 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 pci.ids all 0.0~2022.01.22-1ubuntu0.1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (314 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <>

In [ ]:
# 2. Load All Models
import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from faster_whisper import WhisperModel
import opensmile
import os
from google.colab import userdata
# --- MEMORY CLEANUP ---
gc.collect()
try:
    torch.cuda.empty_cache()
except:
    pass
print("🧹 Memory cleared.")
# ----------------------
# 1. VERIFY TOKEN
from google.colab import userdata
import getpass
try:
    hf_token = userdata.get('HF_TOKEN')
    print("✅ HF_TOKEN found in Secrets.")
except:
    print("⚠️ HF_TOKEN not found in Secrets.")
    print("👉 Please enter your Hugging Face Token below (or press Enter if using a public model):")
    hf_token = getpass.getpass("HF Token: ")
if not hf_token:
    print("⚠️ Warning: No Token provided. Gated models (Mistral) will fail to load.")
# 2. LOADER: WHISPER (STT)
print("⏳ Loading Whisper STT...")
try:
    stt_model = WhisperModel("medium", device="cuda", compute_type="float16")
    print("✅ Whisper (Medium) Loaded on GPU")
except:
    print("⚠️ GPU Error. Fallback to Small/Int8")
    stt_model = WhisperModel("small", device="cuda", compute_type="int8")
# 3. LOADER: OPENSMILE (Feeling)
print("⏳ Loading OpenSMILE...")
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals,
)
# 4. LOADER: MISTRAL (Thinking)
print("⏳ Loading Mistral 7B + 911 Adapter...")
base_model_id = "mistralai/Mistral-7B-v0.1"
adapter_id = "spikecodes/ai-911-operator"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
try:
    model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        quantization_config=bnb_config,
        device_map="auto",
        token=hf_token
    )
    model = PeftModel.from_pretrained(model, adapter_id, token=hf_token)
    tokenizer = AutoTokenizer.from_pretrained(base_model_id, token=hf_token)
    tokenizer.pad_token = tokenizer.eos_token
    print("✅ Mistral Loaded successfully!")
except Exception as e:
    print(f"❌ Mistral Failed to Load: {e}")
print("✅ All Systems Online.")

🧹 Memory cleared.
⚠️ HF_TOKEN not found in Secrets.
👉 Please enter your Hugging Face Token below (or press Enter if using a public model):
HF Token: ··········
⏳ Loading Whisper STT...
✅ Whisper (Medium) Loaded on GPU
⏳ Loading OpenSMILE...
⏳ Loading Mistral 7B + 911 Adapter...


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

✅ Mistral Loaded successfully!
✅ All Systems Online.


In [ ]:
# 3. Server Logic (Colab Brain)
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import nest_asyncio
import shutil
import json
import time
import subprocess
import threading
import re
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
nest_asyncio.apply()
@app.post("/process_emergency")
async def process_emergency(audio: UploadFile = File(...)):
    start_time = time.time()
    # A. Save & Convert
    raw_filename = f"raw_{audio.filename}"
    clean_filename = "clean_input.wav"
    with open(raw_filename, "wb") as buffer:
        shutil.copyfileobj(audio.file, buffer)

    subprocess.run([
        "ffmpeg", "-y", "-i", raw_filename,
        "-ar", "16000", "-ac", "1", clean_filename
    ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    # B. Transcribe
    print("🔊 Transcribing...")
    segments, info = stt_model.transcribe(clean_filename, beam_size=5)
    transcript = " ".join([segment.text for segment in segments])
    print(f"🗣️ Transcript: {transcript}")
    # C. Emotion
    features = smile.process_file(clean_filename)
    pitch_mean = features['F0semitoneFrom27.5Hz_sma3nz_amean'].values[0]
    loudness_mean = features['loudness_sma3_amean'].values[0]
    emotion_tag = "Calm"
    if pitch_mean > 35 or loudness_mean > 2.0:
        emotion_tag = "Panic"
    print(f"💓 Emotion: {emotion_tag}")
    # D. Mistral Generation
    print("🧠 Thinking...")
    prompt = f"911 Operator: 9-1-1, what's your emergency?\nCaller: {transcript}\n911 Operator:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    # Hallucination Fix (Stop before "Caller:")
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_part = full_output.split("911 Operator:")[-1]
    response_text = response_part.split("Caller:")[0].strip()
    print(f"🤖 Response: {response_text}")
    return JSONResponse(content={
        "transcript": transcript,
        "response": response_text,
        "acoustic_emotion": emotion_tag
    })
# --- START TUNNEL ---
def start_cloudflared():
    process = subprocess.Popen(['/tmp/cloudflared', 'tunnel', '--url', 'http://localhost:8000'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in iter(process.stderr.readline, b''):
        line = line.decode('utf-8')
        if 'trycloudflare.com' in line:
            try:
                url = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line).group(0)
                print(f"\n🔗 CONNECT TO THIS URL: {url}\n")
            except:
                print(f"\n🔗 FOUND URL IN LINE: {line.strip()}\n")
threading.Thread(target=start_cloudflared, daemon=True).start()
time.sleep(3)
# --------------------
print("🚀 Starting Server...")
config = uvicorn.Config(app, host="0.0.0.0", port=8000)
server = uvicorn.Server(config)
await server.serve()


🔗 FOUND URL IN LINE: 2026-01-01T03:51:58Z INF Requesting new quick Tunnel on trycloudflare.com...

🚀 Starting Server...


INFO:     Started server process [226]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)



🔗 CONNECT TO THIS URL: https://resolve-consumption-occur-getting.trycloudflare.com

INFO:     175.107.216.135:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     175.107.216.135:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     175.107.216.135:0 - "GET /docs HTTP/1.1" 200 OK
🔊 Transcribing...
🗣️ Transcript:  هالب هالب
💓 Emotion: Calm
🧠 Thinking...
🤖 Response: 9-1-1, what's your emergency?
INFO:     175.107.216.135:0 - "POST /process_emergency HTTP/1.1" 200 OK
🔊 Transcribing...
🗣️ Transcript:  There is a fire.
💓 Emotion: Calm
🧠 Thinking...
🤖 Response: What's the address?
INFO:     175.107.216.135:0 - "POST /process_emergency HTTP/1.1" 200 OK
🔊 Transcribing...
🗣️ Transcript:  Thanks for watching!
💓 Emotion: Calm
🧠 Thinking...
🤖 Response: What's your emergency?
INFO:     175.107.216.135:0 - "POST /process_emergency HTTP/1.1" 200 OK
🔊 Transcribing...
🗣️ Transcript:  There is a fire.
💓 Emotion: Calm
🧠 Thinking...
🤖 Response: What's the address?
INFO:     175.107.216.135:0 - "POST /process_emergency HTTP/1.1